In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
path = '/kaggle/input/kazakhstan-respa-final-day-2-late-competition/'
train_sentences = pd.read_csv(path + 'train_sentences.csv', parse_dates=['submitted_date'])
from sklearn.utils import shuffle
train_sentences = shuffle(train_sentences, random_state=42)
mn = pd.Timestamp('2000-01-01 00:00:00')
train_sentences['seconds'] = (train_sentences['submitted_date'] - mn).dt.total_seconds().astype('int64')
train_sentences = train_sentences.drop(columns='submitted_date')
train_sentences['sentences'] += ' '
train_sentences['sentences'] += train_sentences['full_categories']
train_sentences = train_sentences.drop(columns='full_categories')
def word_clean(x):
    x = x.lower()
    return x
def sentence_clean(st):
    word = st.split()
    res = ""
    for a in word:
        res += word_clean(a)
        res += ' '
    return res
train_sentences['sentences'] = train_sentences['sentences'].apply(sentence_clean)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import *
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, sublinear_tf=True, lowercase=True), SGDRegressor(random_state = 42, max_iter = 10000))
pipe.fit(train_sentences['sentences'], train_sentences['seconds'])

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True)),
                ('sgdregressor',
                 SGDRegressor(max_iter=10000, random_state=42))])

In [2]:
test_sentences = pd.read_csv(path + 'test_sentences.csv')

test_sentences['first_sentence'] = test_sentences['first_sentence'].apply(sentence_clean)
test_sentences['second_sentence'] = test_sentences['second_sentence'].apply(sentence_clean)
res1 = pipe.predict(test_sentences['first_sentence'])
res2 = pipe.predict(test_sentences['second_sentence'])
ans = pd.read_csv(path + 'sample_submission.csv')
for i in range(len(ans)):
    if res1[i] > res2[i]:
        ans['label'].iloc[i] = 0
    else:
        ans['label'].iloc[i] = 1
ans.to_csv('submission.csv', index = False)
ans

/tmp/ipykernel_13/2056324420.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ans['label'].iloc[i] = 0
/tmp/ipykernel_13/2056324420.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values

,ID,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1
...,...,...
24045,24045,0
24046,24046,0
24047,24047,0
24048,24048,0
